In [25]:
import requests
import pandas as pd
from selenium import webdriver
from scrapy.http import TextResponse
import seaborn as sns
import getpass
import time


###

In [26]:
# 여기서부터 시작

In [34]:
# 로그인 함수 (seleniun webdriver)

def login():
    login_url = "https://www.instagram.com/accounts/login/?source=auth_switcher"

    driver = webdriver.Chrome()
    driver.get(login_url)
    
    my_id = input('id를 입력하세요: ')
    my_password = getpass.getpass('password를 입력하세요: ')

    driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP[type=text]').send_keys(my_id)
    driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP[type=password]').send_keys(my_password)
    driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF[type=submit]').click()
    
    time.sleep(2.5)
    
    # 알림설정 하라는 modal창이 뜨는경우 '나중에 하기'를 클릭하는 코드
    alert_modal = driver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div')
    if alert_modal:
        driver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div > div.mt3GC > button.aOOlW.HoLwm').click()
    
    
    

In [28]:
# 검색어를 입력하고 게시물이 가장 많은 검색결과를 클릭 (seleniun webdriver)

def crawling_keyword(word):
    keyword = word

    driver.find_element_by_css_selector('#react-root > section > nav > div._8MQSO.Cx7Bp > div > div > div.LWmhU._0aCwM > input').send_keys(keyword)

    # 키워드 입력하고, 키워드에 대한 검색 리스트가 뜨는데 걸리는 시간만큼 기다려준다
    time.sleep(1.5)
    
    search_list = driver.find_elements_by_css_selector('#react-root > section > nav > div._8MQSO.Cx7Bp > div > div > div.LWmhU._0aCwM > div:nth-child(4) > div.drKGC > div > a')
    
    # 서칭된 a 엘리먼트들 중에 게시물 갯수가 가장 많은 a 엘리먼트를 뽑는다
    number_list = []

    for element in search_list:
        splited_element = element.text.split('게시물')
        if len(splited_element) == 2:
            number_list.append(int(re.sub(",", "", splited_element[1])))
        else:
            number_list.append(0)

    max_number = max(number_list)
    click_index = number_list.index(max_number)
    
    # 클릭하고 렌더링이 완료되도록 조금 기다리기
    search_list[click_index].click()
    time.sleep(1)


In [29]:
# 인기게시물 9개는 빼고, 최근사진부터 크롤링 시작(인기게시물과 최근게시물에 중복 게시물이 있을거 같아서)

In [30]:
# 처음 렌더링된 화면의 24개의 게시물을 크롤링하는 함수

def initial_crawling():
    urls = driver.find_elements_by_xpath(
        '//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
    urls = [url.get_attribute("href") for url in urls]

    hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])

    for i in range(len(urls)):
        req = requests.get(urls[i])
        response = TextResponse(req.url, body=req.text, encoding="utf-8")
        hash_tag_list = response.css("meta[property='instapp:hashtags']")
        hash_tag_list = ['#' + hash_tag.attrib['content']
                         for hash_tag in hash_tag_list]
        hash_tags = ', '.join(hash_tag_list)

        data = [
            {'time': 0, 'hash_tag': hash_tags}
        ]
        df = pd.DataFrame(data)
        hash_tag_df = hash_tag_df.append(df)

    return hash_tag_df.reset_index().drop(columns=['index'])

In [31]:
# 스크롤내리면서 12개씩 추가되는 게시물을 크롤링하는 함수 (두번째만 실행) - chrome 전체화면 기준
# 현재 브라우저 화면 크기에 따라 추가적으로 불러오는 div 갯수가 달라짐
# 전체화면 기준 div의 urls 총 갯수 = 24 -> 36 -> 45 -> 45 -> ...

def second_crawling():
    for i in range(0, 1):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        time.sleep(1)

        urls = driver.find_elements_by_xpath(
            '//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
        urls = [url.get_attribute("href") for url in urls]

        hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])

        for i in range(len(urls) - 12, len(urls)):
            req = requests.get(urls[i])
            response = TextResponse(req.url, body=req.text, encoding="utf-8")
            hash_tag_list = response.css("meta[property='instapp:hashtags']")
            hash_tag_list = ['#' + hash_tag.attrib['content']
                             for hash_tag in hash_tag_list]
            hash_tags = ', '.join(hash_tag_list)

            data = [
                {'time': 0, 'hash_tag': hash_tags}
            ]
            df = pd.DataFrame(data)
            hash_tag_df = hash_tag_df.append(df)

    return hash_tag_df.reset_index().drop(columns=['index'])

In [32]:
# 세번째부터는 이 함수 반복하기 - 게시물 9개씩 크롤링

def repeat_crawling():
    for i in range(0, 1):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        time.sleep(1)

        urls = driver.find_elements_by_xpath(
            '//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
        urls = [url.get_attribute("href") for url in urls]

        hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])

        for i in range(len(urls) - 9, len(urls)):
            req = requests.get(urls[i])
            response = TextResponse(req.url, body=req.text, encoding="utf-8")
            hash_tag_list = response.css("meta[property='instapp:hashtags']")
            hash_tag_list = ['#' + hash_tag.attrib['content']
                             for hash_tag in hash_tag_list]
            hash_tags = ', '.join(hash_tag_list)

            data = [
                {'time': 0, 'hash_tag': hash_tags}
            ]
            df = pd.DataFrame(data)
            hash_tag_df = hash_tag_df.append(df)

    return hash_tag_df.reset_index().drop(columns=['index'])

In [87]:
# %%writefile insta_crawling.py
# import requests
# import pandas as pd
# from selenium import webdriver
# from scrapy.http import TextResponse
# import seaborn as sns
# import getpass
# import time


class instagram_crawling():
    
    driver = ""
    
    def __init__(self):
        login_url = "https://www.instagram.com/accounts/login/?source=auth_switcher"
        self.driver = webdriver.Chrome()
        self.driver.get(login_url)
    
    
    # 로그인 함수 (seleniun webdriver)
    def login(self):

        my_id = input('id를 입력하세요: ')
        my_password = getpass.getpass('password를 입력하세요: ')

        self.driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP[type=text]').send_keys(my_id)
        self.driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP[type=password]').send_keys(my_password)
        self.driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF[type=submit]').click()

        time.sleep(3)

        # 알림설정 하라는 modal창이 뜨는경우 '나중에 하기'를 클릭하는 코드
        alert_modal = self.driver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div')
        if alert_modal:
            self.driver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div > div.mt3GC > button.aOOlW.HoLwm').click()




    # 검색어를 입력하고 게시물이 가장 많은 검색결과를 클릭 (seleniun webdriver)
    def crawling_keyword(self, word):
        keyword = word

        self.driver.find_element_by_css_selector('#react-root > section > nav > div._8MQSO.Cx7Bp > div > div > div.LWmhU._0aCwM > input').send_keys(keyword)

        # 키워드 입력하고, 키워드에 대한 검색 리스트가 뜨는데 걸리는 시간만큼 기다려준다
        time.sleep(1.5)

        search_list = self.driver.find_elements_by_css_selector('#react-root > section > nav > div._8MQSO.Cx7Bp > div > div > div.LWmhU._0aCwM > div:nth-child(4) > div.drKGC > div > a')

        # 서칭된 a 엘리먼트들 중에 게시물 갯수가 가장 많은 a 엘리먼트를 뽑는다
        number_list = []

        for element in search_list:
            splited_element = element.text.split('게시물')
            if len(splited_element) == 2:
                number_list.append(int(re.sub(",", "", splited_element[1])))
            else:
                number_list.append(0)

        max_number = max(number_list)
        click_index = number_list.index(max_number)

        # 클릭하고 렌더링이 완료되도록 조금 기다리기
        search_list[click_index].click()
        time.sleep(1.5)




    # 처음 렌더링된 화면의 24개의 게시물을 크롤링하는 함수
    def initial_crawling(self):
        urls = self.driver.find_elements_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
        urls = [url.get_attribute("href") for url in urls]

        hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])

        for i in range(len(urls)):
            req = requests.get(urls[i])
            response = TextResponse(req.url, body=req.text, encoding="utf-8")
            hash_tag_list = response.css("meta[property='instapp:hashtags']")
            hash_tag_list = ['#' + hash_tag.attrib['content'] for hash_tag in hash_tag_list]
            hash_tags = ', '.join(hash_tag_list)

            data = [
                {'time': 0, 'hash_tag': hash_tags}
            ]
            df = pd.DataFrame(data)
            hash_tag_df = hash_tag_df.append(df)

        return hash_tag_df.reset_index().drop(columns=['index'])




    # 스크롤내리면서 12개씩 추가되는 게시물을 크롤링하는 함수 (두번째만 실행) - chrome 전체화면 기준
    # 현재 브라우저 화면 크기에 따라 추가적으로 불러오는 div 갯수가 달라짐
    # 전체화면 기준 div의 urls 총 갯수 = 24 -> 36 -> 45 -> 45 -> ...
    def second_crawling(self):
        for i in range(0, 1):
            self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            time.sleep(1.5)

            urls = self.driver.find_elements_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
            urls = [url.get_attribute("href") for url in urls]

            hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])

            for i in range(len(urls) - 12, len(urls)):
                req = requests.get(urls[i])
                response = TextResponse(req.url, body=req.text, encoding="utf-8")
                hash_tag_list = response.css("meta[property='instapp:hashtags']")
                hash_tag_list = ['#' + hash_tag.attrib['content'] for hash_tag in hash_tag_list]
                hash_tags = ', '.join(hash_tag_list)

                data = [
                    {'time': 0, 'hash_tag': hash_tags}
                ]
                df = pd.DataFrame(data)
                hash_tag_df = hash_tag_df.append(df)

        return hash_tag_df.reset_index().drop(columns=['index'])



    # 세번째부터는 이 함수 반복하기 - 게시물 9개씩 크롤링
    def repeat_crawling(self):
        for i in range(0, 1):
            self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            time.sleep(1.5)

            urls = self.driver.find_elements_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
            urls = [url.get_attribute("href") for url in urls]

            hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])

            for i in range(len(urls) - 9, len(urls)):
                req = requests.get(urls[i])
                response = TextResponse(req.url, body=req.text, encoding="utf-8")
                hash_tag_list = response.css("meta[property='instapp:hashtags']")
                hash_tag_list = ['#' + hash_tag.attrib['content'] for hash_tag in hash_tag_list]
                hash_tags = ', '.join(hash_tag_list)

                data = [
                    {'time': 0, 'hash_tag': hash_tags}
                ]
                df = pd.DataFrame(data)
                hash_tag_df = hash_tag_df.append(df)

        return hash_tag_df.reset_index().drop(columns=['index'])


In [81]:
insta = instagram_crawling()

In [82]:
insta.login()

id를 입력하세요: yeejun90
password를 입력하세요: ········


In [83]:
insta.crawling_keyword('초코우유')

In [84]:
insta.initial_crawling()

time                                           hash_tag
0     0  #당충전, #울산, #제이제이커피, #울산카페, #금요일, #디저트, #수제간식, ...
1     0  #초코우유, #제주말차, #오레오, #인절미, #라임, #오렌지요거트, #산딸기요거...
2     0  #사이좋은남매, #choco, #비숑, #우다다다, #poodlestagram, #...
3     0  #인친, #like, #instagram, #seoul, #핫플, #홍대, #데일리...
4     0  #펭럽유, #딸기우유, #youus, #펭수, #ebs, #giantpengtv, ...
5     0                                              #초코우유
6     0                     #초코아이스크림, #초코우유, #초코바라기, #초코요정
7     0                                  #김치, #초코우유, #짜파게티
8     0                                         #초코우유, #06
9     0  #꽃중독, #내공간, #아이스초코, #초코중독, #집, #제티, #핫초코미떼, #봄...
10    0  #안경나라맞은편, #티라미수, #황치즈, #음성터미널, #마카롱예약, #음성마카롱,...
11    0  #취미, #홈카페, #맞팔, #소통, #초코우유, #following, #likef...
12    0  #초코우유, #원당카페, #원당늘봄, #인천카페, #검단카페, #마전카페, #당하카...
13    0  #미운, #직장인소확행, #소확행_작지만_확실한_행복🌿🌿, #간식거리, #초코바, ...
14    0  #오레오케이크, #초코우유, #간식스타그램, #허쉬초콜릿, #초콜릿, #라떼맛집, ...
15    0  #럽스타그램, #첫줄반사, #첫줄안녕, #초코우유, #좋반환영, #선팔환영, #lo...
16    0  #온도차, #토이푸들, #세이블푸들, #poodle, #쿨견, #푸숑스타그램, #푸...
17    0  #덴마크초코초코우유, #초코우유, #팝콘, #편의점, #우유, #편스타그램, #세븐...
18    0                #딸기, #초코우유, #요플레, #비요뜨, #바나나, #몰티져스
19    0  #가족, #푸들, #래브라도리트리버, #반려견, #소형견, #대형견, #댕댕이, #...
20    0  #당충전, #집앞산, #오늘은뭐하고놀까, #이게무슨짓, #초코우유, #가나초코우유,...
21    0  #초코우유, #카카오랏, #쪼꼬우유, #달다구리, #애들간식, #카카오밀크, #스페...
22    0  #유어스, #지에스25, #펭럽유, #펭럽유초코, #펭수우유, #초코우유, #gs2...
23    0  #초코, #팔로우, #맞팔해요, #엄마_고마워❤️, #선팔, #파스타, #맞팔, #...

In [85]:
insta.second_crawling()

time                                           hash_tag
0     0  #졸업사진, #매매종목, #섹스스토리, #문의가능, #점심레스토랑, #선플, #세종...
1     0  #인친환영, #선팔하면맞팔가요, #소통해요, #선팔맞팔, #오늘의간식, #댓글환영,...
2     0                                                   
3     0  #탄생화컵, #생일선물, #🍫, #직접만든, #홈카페, #초코우유, #데이지, #홈...
4     0  #꼭, #사랑입니다그램, #중요치않아, #어피치는, #어피치덕후, #중요하다그램, ...
5     0                #gs25, #초코우유, #펭럽유초코우유, #펭수, #펭럽유초코
6     0                                                   
7     0                    #리얼쇼콜라, #마카롱꼬끄, #초코우유, #스믈여덟마카롱
8     0  #다이어트, #좋아요, #selfie, #럽스타그램, #건강스타그램, #다이어트일기...
9     0  #파주카페추천, #헤이리마을, #헤이리카페, #일산맘, #캐릭터디자인, #김포카페,...
10    0                    #불닭떡볶이, #언니와, #내돈으로, #초코우유, #어제
11    0  #🐧, #먹스타, #insta, #초코우유, #우유, #팔로우, #펭수, #펭럽유,...

In [86]:
insta.repeat_crawling()

time                                           hash_tag
0    0  #미쁜카페, #한옥, #쑥쑥라떼, #제주리얼말차라떼, #병음료, #제주도, #맛있다...
1    0  #카페선호하다, #dessert, #에그타르트, #일상, #👊🏻, #롤케이크, #🍫...
2    0  #일상, #좋아하는것도다다름, #아들둘맘, #도너츠, #바나나우유, #두아들맘, #...
3    0  #초코우유, #오레오, #솔티카라멜, #초코파운드케익, #macaron, #콘치즈휘...
4    0                        #초코우유, #칼로리바란스, #앤드, #김남길생각
5    0  #망원동까눌레, #단호박우유, #밀크티맛집, #까눌레, #카페망원정, #생딸기우유,...
6    0           #아들몰래먹기, #엄마스트레스해소용, #꿀맛💕, #미안해아들, #초코우유
7    0                                                   
8    0  #오레오레오, #누가바, #인절미, #티라미수, #미니마카롱, #순수우유, #머랭쿠...

In [21]:
import pprint

In [17]:
# 맨처음 화면의 게시물 갯수

urls = driver.find_elements_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
urls = [url.get_attribute("href") for url in urls]
len(urls)

24

In [577]:
# 스크롤을 한번 끝까지 내렸을때 새로 불러온 게시물을 포함한 전체 게시물 갯수

driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
time.sleep(1.5)
urls = driver.find_elements_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
urls = [url.get_attribute("href") for url in urls]
len(urls)

45

In [18]:
req = requests.get(urls[0])
req.url

'https://www.instagram.com/p/B9pxuKenKyG/'

In [19]:
response = TextResponse(req.url, body=req.text, encoding="utf-8")

response

<200 https://www.instagram.com/p/B9pxuKenKyG/>

In [78]:
# pprint.pprint(response.text)

In [581]:
response.xpath('//*[@id="react-root"]/section/main/div/div[1]/article/div[2]/div[1]/ul/div/li/div/div/div[2]/span/a')

[]

In [79]:
# pprint.pprint(req.text)

In [32]:
driver.quit()